Das Script liest Datein der Art "Analytics Alle Websitedaten Audience Overview 20220801-20220831 20210801-20210831" ein,<br>
wandelt das Datum ins ISO-Format um, führt die drei Einzeldateien für 'Users', 'Sessions' und Pageviews zusammen, schreibt sie in eine CSV-Datei und erstellt am Ende eine Pivot-Tabelle mit den Monatsdaten

In [1]:
import pandas as pd
import numpy as np
#pd.__version__

In [2]:
files = ["Sessions","Users","Pageviews"]
filename_template = '{} 20210101-20211014.csv'
dataframes = []

In [3]:
for i in range (0,len(files)):
    csv_data_file = filename_template.format(files[i])
    cols = ['Day Index',files[i]]
    print(csv_data_file,cols)
    df_tmp = pd.read_csv(csv_data_file,sep=',',header=0, usecols=cols, parse_dates=[0], index_col=0,thousands=',')
    df_tmp[files[i]] = df_tmp[files[i]].astype(int)
    dataframes += [df_tmp]

Sessions 20210101-20211014.csv ['Day Index', 'Sessions']
Users 20210101-20211014.csv ['Day Index', 'Users']
Pageviews 20210101-20211014.csv ['Day Index', 'Pageviews']


In [4]:
df_merged = pd.merge(
        pd.merge(dataframes[0],
                 dataframes[1],
                 left_index=True,
                 right_index=True
                ),
                 dataframes[2],
                 left_index=True,
                 right_index=True
                )

In [5]:
#dataframes
#df_merged


In [6]:
df_merged['Year']  = df_merged.index.year.astype('category')
df_merged['Month'] = df_merged.index.month.astype('category')
df_merged.dtypes

Sessions        int64
Users           int64
Pageviews       int64
Year         category
Month        category
dtype: object

In [7]:
df_merged.to_csv('Consolidated 20210101-20211014.csv')

In [8]:
pivot = pd.pivot_table(df_merged, 
                       index=['Month'], 
                       columns=['Year'],
                       values=["Users","Sessions","Pageviews"],
                       aggfunc = 'sum', 
                       fill_value=0, margins=False
                      )

In [9]:
pivot.dtypes

           Year
Pageviews  2021    int64
           2022    int64
Sessions   2021    int64
           2022    int64
Users      2021    int64
           2022    int64
dtype: object

In [10]:
pivot

Pageviews          Sessions            Users       
Year       2021     2022     2021    2022     2021   2022
Month                                                    
1       9245276  2619203  2390611  261917  1640285  26189
2       9205413        0  2276599       0  1560994      0
3       9973808        0  2466865       0  1679733      0
4       9778438        0  2415951       0  1669745      0
5      11028055        0  2615205       0  1778502      0
6      11101271        0  2570095       0  1770023      0
7      11697018        0  2668631       0  1811710      0
8       9916761        0  2713303       0  1906870      0
9      10574801        0  2690550       0  1916595      0
10      5378804        0  1307165       0   946210      0

In [11]:
pivot['Pageviews'][2021][1]

9245276

In [12]:
pivot['Sessions'][2022][1]

261917

In [13]:
import json

In [14]:
#pivot.columns = pd.Index(list(pivot.columns))
#pivot = pivot.reset_index()

Der Pandas JSON-Export produziert im vorliegenden Fall ungültiges JSON

In [15]:
pivot.to_csv('pivot.csv')

Mit diesem Code lässt sich ein Output als valides JSON erreichen

In [21]:
p2d = pivot.to_dict()
jsonout = {}
for k in p2d:
    new_k = '-'.join(map(lambda x:str(x),list(k)))
    jsonout[new_k] = p2d[k]
json.dumps(jsonout)

'{"Pageviews-2021": {"1": 9245276, "2": 9205413, "3": 9973808, "4": 9778438, "5": 11028055, "6": 11101271, "7": 11697018, "8": 9916761, "9": 10574801, "10": 5378804}, "Pageviews-2022": {"1": 2619203, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0}, "Sessions-2021": {"1": 2390611, "2": 2276599, "3": 2466865, "4": 2415951, "5": 2615205, "6": 2570095, "7": 2668631, "8": 2713303, "9": 2690550, "10": 1307165}, "Sessions-2022": {"1": 261917, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0}, "Users-2021": {"1": 1640285, "2": 1560994, "3": 1679733, "4": 1669745, "5": 1778502, "6": 1770023, "7": 1811710, "8": 1906870, "9": 1916595, "10": 946210}, "Users-2022": {"1": 26189, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0}}'

#### Hinweis
pivot.to_dict() benutzt Tuples als Keys und gibt diese bei der Umwandlung in JSON falsch aus

In [20]:
p2d

{('Pageviews', 2021): {1: 9245276,
  2: 9205413,
  3: 9973808,
  4: 9778438,
  5: 11028055,
  6: 11101271,
  7: 11697018,
  8: 9916761,
  9: 10574801,
  10: 5378804},
 ('Pageviews', 2022): {1: 2619203,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0},
 ('Sessions', 2021): {1: 2390611,
  2: 2276599,
  3: 2466865,
  4: 2415951,
  5: 2615205,
  6: 2570095,
  7: 2668631,
  8: 2713303,
  9: 2690550,
  10: 1307165},
 ('Sessions', 2022): {1: 261917,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0},
 ('Users', 2021): {1: 1640285,
  2: 1560994,
  3: 1679733,
  4: 1669745,
  5: 1778502,
  6: 1770023,
  7: 1811710,
  8: 1906870,
  9: 1916595,
  10: 946210},
 ('Users', 2022): {1: 26189,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0}}